In [1]:
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
client.models.list()

SyncPage[Model](data=[Model(id='gemma-3-27b-it', created=None, object='model', owned_by='organization_owner'), Model(id='phi-3.5-vision-instruct', created=None, object='model', owned_by='organization_owner'), Model(id='text-embedding-nomic-embed-text-v1.5@q4_k_m', created=None, object='model', owned_by='organization_owner'), Model(id='llama-3.3-70b-instruct', created=None, object='model', owned_by='organization_owner'), Model(id='qwen2-7b-instruct', created=None, object='model', owned_by='organization_owner'), Model(id='meta-llama-3.1-70b-instruct', created=None, object='model', owned_by='organization_owner'), Model(id='mistral-7b-instruct-v0.3', created=None, object='model', owned_by='organization_owner'), Model(id='llama-3-8b-instruct-64k', created=None, object='model', owned_by='organization_owner'), Model(id='gemma-2-27b-it', created=None, object='model', owned_by='organization_owner'), Model(id='meta-llama-3.1-8b-instruct', created=None, object='model', owned_by='organization_owne

In [11]:
import os
from PIL import Image, ImageOps
import pandas as pd

# Load data
df = pd.read_csv("votes_clean.csv")

image_dir = "gsv/final_photo_dataset"
output_dir = "merged_images"
os.makedirs(output_dir, exist_ok=True)

separator_width = 20
separator_color = (255, 255, 255)

def get_image_path(base_name):
    for ext in [".JPG", ".jpg", ".jpeg"]:
        path = os.path.join(image_dir, f"{base_name}{ext}")
        if os.path.exists(path):
            return path
    return None

# Track metadata
matched_rows = []
missing_logs = []

merged_index = 0
row_idx = 0

while merged_index < 100 and row_idx < len(df):
    row = df.iloc[row_idx]
    row_idx += 1

    left_id = str(row["left"])
    right_id = str(row["right"])

    left_path = get_image_path(left_id)
    right_path = get_image_path(right_id)

    if not left_path or not right_path:
        if not left_path:
            missing_logs.append(f"Row {row_idx - 1}: Missing left image '{left_id}'")
        if not right_path:
            missing_logs.append(f"Row {row_idx - 1}: Missing right image '{right_id}'")
        continue  # Skip this row

    try:
        left_img = Image.open(left_path)
        right_img = Image.open(right_path)

        max_height = max(left_img.height, right_img.height)
        left_img = ImageOps.pad(left_img, (left_img.width, max_height))
        right_img = ImageOps.pad(right_img, (right_img.width, max_height))

        separator = Image.new("RGB", (separator_width, max_height), separator_color)

        merged = Image.new("RGB", (left_img.width + separator_width + right_img.width, max_height))
        merged.paste(left_img, (0, 0))
        merged.paste(separator, (left_img.width, 0))
        merged.paste(right_img, (left_img.width + separator_width, 0))

        filename = f"merged_{merged_index:03d}.jpg"
        merged.save(os.path.join(output_dir, filename))

        matched_rows.append({
            "merged_index": merged_index,
            "row_index": row_idx - 1,
            "left": left_id,
            "right": right_id,
            "study_question": row["study_question"],
            "choice": row["choice"]
        })

        merged_index += 1

    except Exception as e:
        missing_logs.append(f"Row {row_idx - 1}: ERROR during merge: {e}")

# Save metadata
pd.DataFrame(matched_rows).to_csv("merged_metadata.csv", index=False)

# Print missing log
print("✅ Merged images saved to 'merged_images/'")
print("✅ Metadata saved to 'merged_metadata.csv'")
print("⚠️ Missing or error rows:")
for log in missing_logs:
    print(log)


✅ Merged images saved to 'merged_images/'
✅ Metadata saved to 'merged_metadata.csv'
⚠️ Missing or error rows:
Row 4: Missing left image '513d56fefdc9f035870030ad'
Row 15: Missing right image '513d9f1efdc9f03587008639'
Row 81: Missing left image '513d56fefdc9f035870030ad'
